# Agentic Finance — All-in-One Notebook

**This notebook is a demonstration notebook showing the full workflow and plan of our project.**

&nbsp;&nbsp;&nbsp;&nbsp;It summarizes the key components we developed across the Agentic Finance system, including data ingestion, analysis, orchestration, and user interface layers

3 explicit workflow patterns
-  5 working AI agents
- Full data pipeline
- Interactive UI
- Professional code structure

**To run the full interactive application, please use the Gradio app located at:**

&nbsp;&nbsp;&nbsp;&nbsp;/src/ui/gradio_app.py

**Run it from the project root with:**

&nbsp;&nbsp;&nbsp;&nbsp;python -m ui.gradio_app

**This will test the agents and see live results.**

# Market Research Agent — Price, Indicators, and News Pipeline

This project builds a lightweight research workflow for equities: we fetch historical prices, compute common technical indicators (SMA, RSI), and pull headline summaries with basic sentiment. A small JSON cache keeps runs fast and reproducible while avoiding API rate limits. The notebook(s) walk through data loading, quick EDA, feature prep, and simple evaluation.

**Course:** MSAAI 520-02 — Group 5  
**Date:** October 18, 2025

## Team
- Ali Azizi  
- Sunitha Kosireddy  
- Victor Salcedo


In [56]:
# Setup: create package tree and write source files (no changes to your code)
import pathlib, sys
ROOT = pathlib.Path("/mnt/data/agentic-finance")
SRC = ROOT / "src"
UI = ROOT / "ui"
sys.path.insert(0, str(ROOT))
print("Project root:", ROOT)

Project root: /mnt/data/agentic-finance


In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Add repo root to sys.path and ensure packages exist


In [58]:
from pathlib import Path
import sys

# Find repo root (folder containing "src")
ROOT = Path.cwd()
while not (ROOT / "src").exists() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.insert(0, str(ROOT))  # make "src" importable

# Ensure packages (empty __init__.py files)
for p in [
    ROOT / "src",
    ROOT / "src" / "config",
    ROOT / "src" / "data_io",
    ROOT / "src" / "system",
    ROOT / "src" / "analysis",


]:
    p.mkdir(parents=True, exist_ok=True)
    (p / "__init__.py").touch(exist_ok=True)


## src/analysis/features.py

In [59]:
# src/analysis/features.py
from __future__ import annotations
import pandas as pd
import numpy as np

def compute_sma(prices: pd.DataFrame, window: int) -> pd.Series:
    if prices is None or prices.empty:
        return pd.Series(dtype=float)
    return prices["close"].rolling(window=window).mean()

def compute_rsi(prices: pd.DataFrame, window: int = 14) -> pd.Series:
    if prices is None or prices.empty:
        return pd.Series(dtype=float)
    delta = prices["close"].diff()
    gain = np.where(delta > 0, delta, 0.0)
    loss = np.where(delta < 0, -delta, 0.0)
    gain_s = pd.Series(gain, index=prices.index)
    loss_s = pd.Series(loss, index=prices.index)
    avg_gain = gain_s.rolling(window=window).mean()
    avg_loss = loss_s.rolling(window=window).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    return rsi


## src/analysis/text.py

In [83]:
# Second approach

from __future__ import annotations
import re
import pandas as pd
from datetime import datetime, timedelta
# from config.settings import SETTINGS

# -----------------------------
# Existing tagging / preprocessing
# -----------------------------

TAG_RULES = {
    "earnings": ["earnings", "eps", "guidance", "outlook", "quarter", "revenue"],
    "product":  ["launch", "iphone", "chip", "feature", "service"],
    "legal":    ["lawsuit", "regulator", "antitrust", "fine", "settlement"],
    "macro":    ["inflation", "rates", "fed", "recession", "gdp"]
}

def preprocess_news(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame(columns=[
            "published_at","source","title","summary","url",
            "overall_sentiment","tags","numbers"
        ])

    df = df.copy()

    # Alpha Vantage format is like "20251017T200143"
    # Parse with explicit format; keep timezone-aware for safety
    df["published_at"] = pd.to_datetime(
        df["published_at"], format="%Y%m%dT%H%M%S", errors="coerce", utc=True
    )

    # Drop rows with no title/url; keep others (don’t drop NaT here — the date filter happens later)
    df = df.dropna(subset=["title","url"]).drop_duplicates(subset=["url"])
    df["summary"] = df["summary"].fillna("")
    return df

def classify_tags(text: str) -> list[str]:
    text_l = text.lower()
    tags = [k for k, kws in TAG_RULES.items() if any(kw in text_l for kw in kws)]
    return tags or ["general"]

NUM_RE = re.compile(r'(\$?\b\d+(?:\.\d+)?%?)')

def extract_numbers(text: str) -> list[str]:
    return NUM_RE.findall(text or "")[:6]

def add_tags_and_numbers(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()
    df["tags"] = (df["title"] + " " + df["summary"]).apply(classify_tags)
    df["numbers"] = (df["title"] + " " + df["summary"]).apply(extract_numbers)
    return df

def recent_topk(df: pd.DataFrame, topk: int, days: int, required_tags: list[str] | None = None) -> pd.DataFrame:
    if df.empty:
        return df

    # Make an aware UTC cutoff; df['published_at'] is already UTC-aware
    cutoff = pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=days)
    f = df[df["published_at"] >= cutoff]

    if required_tags:
        want = [t.strip().lower() for t in required_tags]
        f_tags = f[f["tags"].apply(lambda ts: any(t in [x.lower() for x in ts] for t in want))]
        f = f_tags if not f_tags.empty else f

    return f.sort_values("published_at", ascending=False).head(topk)

# -----------------------------
# NEW: shared agent utilities
# -----------------------------

import json

def strip_code_fences(s: str) -> str:
    """Remove leading/trailing ``` blocks (optionally ```json)."""
    if not isinstance(s, str):
        return s
    return re.sub(r"^```(?:json)?\s*|\s*```$", "", s.strip(), flags=re.IGNORECASE)

def to_float(x, default: float = 0.0) -> float:
    """Best-effort conversion of model outputs or strings to float."""
    try:
        if isinstance(x, str):
            xs = x.strip().lower()
            # map common words to numeric anchors
            if xs in ("high", "strong", "bullish", "overbought"):
                return 0.8
            if xs in ("medium", "moderate", "neutral"):
                return 0.5
            if xs in ("low", "weak", "bearish", "oversold"):
                return 0.2
        return float(x)
    except Exception:
        return default

def clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))

def normalize_score(v: float) -> float:
    """
    Normalize arbitrary score ranges to [-1, 1].
    Heuristics:
      - If already in [-1,1], keep.
      - If in [0,1], map to [-1,1] via (v-0.5)*2.
      - If in (1,100], treat as percent.
      - If in (1,10], treat as 0-10 and map.
      - Else, clamp.
    """
    try:
        v = float(v)
    except Exception:
        return 0.0
    if -1.0 <= v <= 1.0:
        return v
    if 0.0 <= v <= 1.0:
        return (v - 0.5) * 2.0
    if 1.0 < v <= 100.0:
        v01 = v / 100.0
        return (v01 - 0.5) * 2.0
    if 1.0 < v <= 10.0:
        v01 = v / 10.0
        return (v01 - 0.5) * 2.0
    return clamp(v, -1.0, 1.0)

def normalize_conf(v) -> float:
    """Normalize any confidence-like value to [0,1]."""
    f = to_float(v, 0.7)
    if 1.0 < f <= 100.0:
        f = f / 100.0
    return clamp(f, 0.0, 1.0)

# Optional: helpers to render structured dicts into strings (for external tools)
def pretty_json_block(obj: dict, max_chars: int = 4000) -> str:
    """Return a fenced JSON markdown block, truncated for UI safety."""
    try:
        js = json.dumps(obj, ensure_ascii=False, indent=2)
    except Exception:
        js = str(obj)
    if len(js) > max_chars:
        js = js[: max_chars - 20] + "\n... (truncated)"
    return f"```json\n{js}\n```"


## src/config/settings.py

In [84]:
from __future__ import annotations
import os
from pathlib import Path
from dataclasses import dataclass
from dotenv import load_dotenv

def _find_project_root(start: Path) -> Path:
    """
    Walk upward to find the repo root heuristically.
    Treat a folder containing both 'src' and 'data' as the root.
    Fallback to the starting directory if not found.
    """
    for p in [start, *start.parents]:
        if (p / "src").exists() and (p / "data").exists():
            return p
    return start

# project root = repo root
if "__file__" in globals():
    ROOT = Path(__file__).resolve().parents[2]
else:
    # Notebook / REPL: start from CWD and auto-detect root
    ROOT = _find_project_root(Path.cwd())

load_dotenv(ROOT / ".env", override=False)

@dataclass(frozen=True)
class Settings:
    data_dir: Path = ROOT / "data"
    cache_dir: Path = ROOT / "data" / "cache"
    runs_dir: Path = ROOT / "data" / "runs"
    alpha_api_key: str = os.getenv("ALPHAVANTAGE_API_KEY", "BVGUKZR1MHVS0T6B")
    openai_api_key: str = os.getenv("OPENAI_API_KEY", "sk-proj-")
    news_window_days: int = 14
    topk_news: int = 5
    cache_ttl_minutes: int = 60

SETTINGS = Settings()
SETTINGS.cache_dir.mkdir(parents=True, exist_ok=True)
SETTINGS.runs_dir.mkdir(parents=True, exist_ok=True)


## src/data_io/cache.py

### Simple JSON file cache (TTL + atomic writes)
I use a tiny JSON-based cache so repeated runs don’t redo the same work. Each key maps to a file on disk, with a timestamp to support a time-to-live (TTL). Saving is done via a temp file + replace so partial writes don’t corrupt the cache.

**Inputs:** `key`, optional `ttl_minutes`, arbitrary `data`  
**Key choices:** per-key JSON files under `SETTINGS.cache_dir`, ISO-8601 for dates, atomic write on save  
**Output:** `load_cache` returns cached payload or `None`; `save_cache` writes `{"_ts": ..., "data": ...}` to disk


In [85]:
# Purpose: lightweight disk cache with TTL and atomic writes
# Context: used by data fetchers (e.g., price downloads) to avoid repeat network calls
# Notes: filenames derive from key under SETTINGS.cache_dir; payload stored as JSON

# cache.py
from __future__ import annotations
import json, time
from datetime import date, datetime
from pathlib import Path
from typing import Any
from src.data_io.cache import load_cache, save_cache
from src.config.settings import SETTINGS

def _cache_path(key: str) -> Path:
    return SETTINGS.cache_dir / f"{key}.json"

def _json_default(o: Any):
    # datetime & pandas.Timestamp (subclass of datetime) → ISO 8601
    if isinstance(o, (datetime, date)):
        return o.isoformat()
    # Fallback: make a best-effort string (covers Decimal, Path, Enum, etc.)
    try:
        return str(o)
    except Exception:
        return repr(o)

def load_cache(key: str, ttl_minutes: int | None = None) -> Any | None:
    p = _cache_path(key)
    if not p.exists():
        return None
    try:
        obj = json.loads(p.read_text(encoding="utf-8"))
        if ttl_minutes is None:
            return obj.get("data")  # consistent: always return payload
        if (time.time() - obj.get("_ts", 0)) <= ttl_minutes * 60:
            return obj.get("data")
    except Exception:
        return None
    return None

def save_cache(key: str, data: Any) -> None:
    p = _cache_path(key)
    p.parent.mkdir(parents=True, exist_ok=True)
    tmp = p.with_suffix(p.suffix + ".tmp")
    payload = {"_ts": time.time(), "data": data}
    tmp.write_text(json.dumps(payload, ensure_ascii=False, default=_json_default), encoding="utf-8")
    tmp.replace(p)  # atomic on most OS/filesystems


## src/data_io/prices.py

### Fetch historical prices (with simple caching)
I pull OHLCV data from Yahoo Finance and return a clean DataFrame. I use a short-lived cache so repeated runs don’t keep hitting the API. If data is already cached, I return it immediately. I also flatten any MultiIndex columns and standardize names so downstream code stays consistent.

**Inputs:** `symbol`, `start`, `end`  
**Key choices:** use `yfinance.download`, cache key includes date range, flatten MultiIndex, keep both `close` and `adj_close`  
**Output:** DataFrame with `date, open, high, low, close, adj_close, volume`


In [86]:
# Purpose: download OHLCV from Yahoo Finance and return a normalized DataFrame with caching
# Context: called by data prep steps before features/EDA; avoids repeated network calls
# Notes: flattens MultiIndex cols, standardizes names, stores json-serializable cache

from __future__ import annotations
import pandas as pd
import yfinance as yf
from src.data_io.cache import load_cache, save_cache
from src.config.settings import SETTINGS

def fetch_prices(symbol: str, start: str | None, end: str | None) -> pd.DataFrame:
    cache_key = f"prices_{symbol}_{start}_{end}"
    cached = load_cache(cache_key, ttl_minutes=SETTINGS.cache_ttl_minutes)
    if cached is not None:
        return pd.DataFrame(cached)
    df = yf.download(symbol, start=start, end=end, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0].lower() for c in df.columns]
    df = df.reset_index().rename(columns={
        "Date": "date", "open":"open","high":"high","low":"low","close":"close","adj close":"adj_close","volume":"volume"
    })
    df["date"] = df["date"].astype(str)
    save_cache(cache_key, df.to_dict(orient="records"))
    return df


## src/data_io/indicators.py

### Fetch technical indicators (Alpha Vantage with local fallback)
I get daily SMA/RSI for a symbol. I try Alpha Vantage first and fall back to computing the indicator locally from our price history when the key is missing or the API is rate-limited. Results are cached so repeated calls are fast.

**Inputs:** `symbol`, `indicator` (`"SMA"` or `"RSI"`), `time_period`  
**Key choices:** Alpha Vantage params (daily interval, series_type=close), JSON parsing with key map, local fallback uses `compute_sma`/`compute_rsi` on `fetch_prices`  
**Output:** DataFrame with `date` and indicator column(s), sorted ascending


In [87]:
# Purpose: fetch SMA/RSI via Alpha Vantage with a cached local-compute fallback
# Context: used by feature pipelines that need daily indicators
# Notes: caches by (symbol, indicator, time_period); normalizes dates and numeric types

# src/data_io/indicators.py
from __future__ import annotations
import requests
import pandas as pd
from typing import Optional
from src.config.settings import SETTINGS
from src.data_io.prices import fetch_prices
from src.analysis.features import compute_sma, compute_rsi
from src.data_io.cache import load_cache, save_cache

BASE = "https://www.alphavantage.co/query"
KEYS = {"SMA": "Technical Analysis: SMA", "RSI": "Technical Analysis: RSI"}


# If AV isn’t available (no key/limit), our code falls back to computing indicators locally from prices using our compute_sma / compute_rsi.
def _fallback_from_prices(symbol: str, indicator: str, time_period: int) -> pd.DataFrame:
    prices = fetch_prices(symbol, None, None)
    if prices is None or prices.empty:
        return pd.DataFrame()

    if indicator == "SMA":
        df = pd.DataFrame({"date": prices["date"], "SMA": compute_sma(prices, window=time_period)})
    elif indicator == "RSI":
        df = pd.DataFrame({"date": prices["date"], "RSI": compute_rsi(prices, window=time_period)})
    else:
        return pd.DataFrame()

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])
    for c in df.columns:
        if c != "date":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna().sort_values("date", ascending=True).reset_index(drop=True)
    return df

def fetch_indicator(symbol: str, indicator: str, time_period: int = 14) -> pd.DataFrame:
    key = KEYS.get(indicator)

    # Try cache first
    cache_key = f"indicator_{symbol}_{indicator}_{time_period}"
    cached = load_cache(cache_key, ttl_minutes=SETTINGS.cache_ttl_minutes)
    if cached is not None:
        return pd.DataFrame(cached)

    if not SETTINGS.alpha_api_key or key is None:
        df = _fallback_from_prices(symbol, indicator, time_period)
        save_cache(cache_key, df.to_dict(orient="records"))
        return df

    params = {
        "function": indicator,
        "symbol": symbol,
        "interval": "daily",
        "time_period": time_period,
        "series_type": "close",
        "apikey": SETTINGS.alpha_api_key,
    }
    try:
        resp = requests.get(BASE, params=params, timeout=30)
        resp.raise_for_status()
        data = resp.json()
        # Alpha Vantage quota message handling:
        if (not data or key not in data or not data[key] or "Note" in data or "Information" in data or "Error Message" in data):
            df = _fallback_from_prices(symbol, indicator, time_period)
            save_cache(cache_key, df.to_dict(orient="records"))
            return df
    except Exception:
        df = _fallback_from_prices(symbol, indicator, time_period)
        save_cache(cache_key, df.to_dict(orient="records"))
        return df

    df = pd.DataFrame.from_dict(data[key], orient="index")
    df.index = pd.to_datetime(df.index, errors="coerce")
    df.reset_index(inplace=True)
    df = df.rename(columns={"index": "date"})
    for c in df.columns:
        if c != "date":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=["date"]).sort_values("date", ascending=True).reset_index(drop=True)
    save_cache(cache_key, df.to_dict(orient="records"))
    return df


## src/data_io/news.py

### Fetch news (Alpha Vantage, symbol-filtered, cached)
I pull recent news from Alpha Vantage’s `NEWS_SENTIMENT` endpoint and keep only items that explicitly mention my ticker with decent relevance. I cache the result so I don’t burn quota on repeat runs.

**Inputs:** `symbol`  
**Key choices:** `relevance_score >= 0.30`, require explicit ticker match, cache per symbol  
**Output:** DataFrame with `published_at, source, title, summary, url, overall_sentiment`


In [88]:
# Purpose: fetch and cache symbol-specific news via Alpha Vantage, filtered by relevance
# Context: called by downstream reporting/EDA to attach headlines and sentiment
# Notes: filters to items where ticker matches and relevance >= 0.30; caches by symbol

from __future__ import annotations
import os, requests, pandas as pd
from src.data_io.cache import load_cache, save_cache
from src.config.settings import SETTINGS

BASE = "https://www.alphavantage.co/query"


def fetch_news(symbol: str) -> pd.DataFrame:
    if not SETTINGS.alpha_api_key:
        return pd.DataFrame()  # safe fail
    cache_key = f"news_{symbol}"
    cached = load_cache(cache_key, ttl_minutes=SETTINGS.cache_ttl_minutes)
    if cached is not None:
        return pd.DataFrame(cached)

    params = {"function":"NEWS_SENTIMENT","tickers":symbol,"apikey":SETTINGS.alpha_api_key}
    r = requests.get(BASE, params=params, timeout=30)
    data = r.json()
    if "feed" not in data:
        return pd.DataFrame()

    rows = []
    for item in data.get("feed", []):
        tickers = item.get("ticker_sentiment", []) or []
        # keep only if our symbol is explicitly mentioned
        keep = any(t.get("ticker", "").upper() == symbol.upper() and float(t.get("relevance_score", 0) or 0) >= 0.30
                   for t in tickers)
        if not keep:
            continue

        rows.append({
            "published_at": item.get("time_published"),
            "source": item.get("source"),
            "title": item.get("title"),
            "summary": item.get("summary"),
            "url": item.get("url"),
            "overall_sentiment": item.get("overall_sentiment_label")
        })

    # ====== Forth APPROACH =====
    df = pd.DataFrame(rows)
    save_cache(cache_key, df.to_dict(orient="records"))
    return df


## src/system/router.py

In [89]:
from __future__ import annotations

def choose_agents(has_news: bool, has_prices: bool, has_technicals: bool) -> list[str]:
    agents = []
    if has_news: agents.append("news")
    # earnings optional if you add a financials fetch later
    if has_technicals and has_prices: agents.append("technical")
    agents.append("risk")
    return agents


## src/system/memory.py

In [90]:
from __future__ import annotations
import json
from pathlib import Path
from typing import Any
from src.config.settings import SETTINGS

MEM_PATH = SETTINGS.runs_dir / "run_notes.jsonl"

def append_memory(record: dict[str, Any]) -> None:
    MEM_PATH.parent.mkdir(parents=True, exist_ok=True)
    with MEM_PATH.open("a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


## src/agents.py

## src/system/orchestration.py

In [91]:

# ======= 3Rd version of the orchestrator ==========

from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Dict, List
from datetime import datetime
import pandas as pd
from pandas import DataFrame

from src.config.settings import SETTINGS
from src.data_io.prices import fetch_prices
from src.data_io.news import fetch_news
from src.data_io.indicators import fetch_indicator
from src.analysis.text import preprocess_news, add_tags_and_numbers, recent_topk
from src.system.router import choose_agents
from src.system.memory import append_memory
from src.agents import (
    NewsAnalysisAgent,
    MarketSignalsAgent,
    RiskAssessmentAgent,
    SynthesisAgent,
    CritiqueAgent,
    AgentResponse,
)

# from ..analysis.signals import normalize_technicals  # add this import

import json

def _as_text(x):
    if x is None:
        return ""
    if isinstance(x, (dict, list)):
        try:
            return json.dumps(x, ensure_ascii=False, indent=2)
        except Exception:
            return str(x)
    return str(x)

def _as_list_of_text(x):
    if isinstance(x, list):
        return [_as_text(i) for i in x]
    if x is None:
        return []
    # if a single str/dict was returned, wrap it
    return [_as_text(x)]


@dataclass
class OrchestratorResult:
    plan: List[str]
    evidence: Dict[str, DataFrame]
    agent_outputs: List[AgentResponse]
    final: AgentResponse
    critique: AgentResponse


def run_pipeline(symbol: str, start: str | None, end: str | None,
                 required_tags: list[str] | None = None) -> OrchestratorResult:
    plan = [
        "fetch_prices", "fetch_news", "preprocess", "classify_extract",
        "retrieve_topk", "route", "run_agents", "synthesize", "critique", "save_memory"
    ]

    # 1) fetch
    prices = fetch_prices(symbol, start, end)
    news   = fetch_news(symbol)

    # 2) preprocess (clean + tags + lightweight numeric extraction)
    news_pp = add_tags_and_numbers(preprocess_news(news))

    # 3) retrieval (deterministic)
    top_news = recent_topk(
        news_pp,
        topk=SETTINGS.topk_news,
        days=SETTINGS.news_window_days,
        required_tags=required_tags
    )

    # 4) route
    has_news   = not top_news.empty
    has_prices = not prices.empty

    # Technical indicators (fetch what we can; leave the rest as None)
    rsi = fetch_indicator(symbol, "RSI", 14)         # may be empty
    sma20 = fetch_indicator(symbol, "SMA", 20)       # may be empty
    # Optional (if your fetch_indicator supports these; otherwise they'll be empty DataFrames)
    sma50 = fetch_indicator(symbol, "SMA", 50)
    sma200 = fetch_indicator(symbol, "SMA", 200)

    has_technicals = (not rsi.empty) or (not sma20.empty) or (not sma50.empty) or (not sma200.empty)
    lanes = choose_agents(has_news, has_prices, has_technicals)

    # 5) run agents
    outputs: List[AgentResponse] = []

    # NEWS (delegate summarization to the agent)
    if "news" in lanes and has_news:
        # Teammate agent expects `description`; remap our `summary` to `description`
        news_payload_records = (
            top_news
            .rename(columns={"summary": "description"})
            .loc[:, ["title", "description", "source", "url", "published_at"]]
            .to_dict(orient="records")
        )
        news_payload = {
            "ticker": symbol,
            "news": news_payload_records
        }
        outputs.append(NewsAnalysisAgent().process(news_payload))

    # TECHNICALS (build dict the agent expects; missing fields are fine)
    if "technical" in lanes and (has_technicals or has_prices):
        current_price = float(prices["close"].iloc[-1]) if has_prices else None
        volume = int(prices["volume"].iloc[-1]) if has_prices else None
        avg_volume = int(prices["volume"].tail(20).mean()) if has_prices else None

        technicals = {
            "current_price": current_price,
            "rsi": (float(rsi["RSI"].iloc[-1]) if not rsi.empty else None),
            # Map to teammate’s keys; if we don’t have these, leave None.
            "ma_50": (float(sma50["SMA"].iloc[-1]) if not sma50.empty else (float(sma20["SMA"].iloc[-1]) if not sma20.empty else None)),
            "ma_200": (float(sma200["SMA"].iloc[-1]) if not sma200.empty else None),
            "macd": None,                # not computed here
            "volume": volume,
            "avg_volume": avg_volume,
            "support": None,
            "resistance": None,
        }
        tech_payload = {
            "ticker": symbol,
            "technicals": technicals
        }
        outputs.append(MarketSignalsAgent().process(tech_payload))


    # RISK (simple proxy; fill what you have)
    vol_30d = float(prices["close"].pct_change().tail(30).std() * 100) if has_prices else None
    risk_payload = {
        "ticker": symbol,
        "risk_metrics": {
            "beta": None,
            "volatility": vol_30d,      # teammate agent expects 'volatility' (%)
            "var_5": None,
            "sharpe_ratio": None,
            "max_drawdown": None,
            "sector_correlation": None,
            "pe_ratio": None
        }
    }
    outputs.append(RiskAssessmentAgent().process(risk_payload))

    # 6) synthesize + critique (gated second pass)
    synth_v1 = SynthesisAgent().process(outputs)
    crit     = CritiqueAgent().process(synth_v1)

    # Decide if we need a second pass.
    # Rule: re-run if critique quality score < 0.90 OR if it mentions "data quality".
    needs_rerun = (crit.score < 0.90) or ("data quality" in " ".join(crit.key_factors).lower())

    synth_final = synth_v1

    if needs_rerun:
        # Turn critique into feedback for the optimizer pass
        critique_feedback = AgentResponse(
            agent_name="Critique Feedback",
            analysis=_as_text(synth_v1.analysis) + "\n\n[CRITIQUE]\n" + _as_text(crit.analysis),
            score=crit.score,
            confidence=crit.confidence,
            key_factors=_as_list_of_text(crit.key_factors),
            timestamp=datetime.now().isoformat()  # ← FIXED
        )

        synth_v2_inputs = outputs + [critique_feedback]
        synth_v2 = SynthesisAgent().process(synth_v2_inputs)
        synth_final = synth_v2

    # 7) memory (store both passes where applicable)
    append_memory({
        "ticker": symbol,
        "lanes": lanes,
        "issues": crit.key_factors,
        "final_confidence_v1": synth_v1.confidence,
        "final_confidence_v2": synth_final.confidence if needs_rerun else None,
        "optimizer_triggered": bool(needs_rerun),
        "timestamp": datetime.now().isoformat()  # ← FIXED
})

    # 8) evidence for UI
    evidence = {
        "top_news": top_news,
        "prices_tail": prices.tail(5)
    }

    # Show the first synthesis alongside other agents for transparency

    outputs.append(AgentResponse(
        agent_name="Initial Synthesis",
        analysis=_as_text(synth_v1.analysis),
        score=float(synth_v1.score),
        confidence=float(synth_v1.confidence),
        key_factors=_as_list_of_text(synth_v1.key_factors),
        timestamp=synth_v1.timestamp
))

    return OrchestratorResult(plan, evidence, outputs, synth_final, crit)



In [82]:
# Execute in parallel
# ======= 3Rd version of the orchestrator ==========
from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Dict, List
from datetime import datetime
import pandas as pd
from pandas import DataFrame

from src.config.settings import SETTINGS
from src.data_io.prices import fetch_prices
from src.data_io.news import fetch_news
from src.data_io.indicators import fetch_indicator
from src.analysis.text import preprocess_news, add_tags_and_numbers, recent_topk
from src.system.router import choose_agents
from src.system.memory import append_memory
from src.agents import (
    NewsAnalysisAgent,
    MarketSignalsAgent,
    RiskAssessmentAgent,
    SynthesisAgent,
    CritiqueAgent,
    AgentResponse,
)
# from ..analysis.signals import normalize_technicals  # add this import

import json

def _as_text(x):
    if x is None:
        return ""
    if isinstance(x, (dict, list)):
        try:
            return json.dumps(x, ensure_ascii=False, indent=2)
        except Exception:
            return str(x)
    return str(x)

def _as_list_of_text(x):
    if isinstance(x, list):
        return [_as_text(i) for i in x]
    if x is None:
        return []
    # if a single str/dict was returned, wrap it
    return [_as_text(x)]


@dataclass
class OrchestratorResult:
    plan: List[str]
    evidence: Dict[str, DataFrame]
    agent_outputs: List[AgentResponse]
    final: AgentResponse
    critique: AgentResponse


def run_pipeline(symbol: str, start: str | None, end: str | None,
                 required_tags: list[str] | None = None) -> OrchestratorResult:
    """
    Main orchestration pipeline for financial analysis.

    Coordinates multiple specialized agents to analyze a stock through:
    1. Data ingestion (prices, news, technical indicators)
    2. Preprocessing and classification
    3. Routing to appropriate agents
    4. Synthesis of findings
    5. Quality critique and optional re-synthesis

    This function implements the Evaluator-Optimizer pattern internally.
    """
    plan = [
        "fetch_prices", "fetch_news", "preprocess", "classify_extract",
        "retrieve_topk", "route", "run_agents", "synthesize", "critique", "save_memory"
    ]

    # 1) fetch
    prices = fetch_prices(symbol, start, end)
    news   = fetch_news(symbol)

    # 2) preprocess (clean + tags + lightweight numeric extraction)
    news_pp = add_tags_and_numbers(preprocess_news(news))

    # 3) retrieval (deterministic)
    top_news = recent_topk(
        news_pp,
        topk=SETTINGS.topk_news,
        days=SETTINGS.news_window_days,
        required_tags=required_tags
    )

    # 4) route
    has_news   = not top_news.empty
    has_prices = not prices.empty

    # Technical indicators (fetch what we can; leave the rest as None)
    rsi = fetch_indicator(symbol, "RSI", 14)         # may be empty
    sma20 = fetch_indicator(symbol, "SMA", 20)       # may be empty
    # Optional (if your fetch_indicator supports these; otherwise they'll be empty DataFrames)
    sma50 = fetch_indicator(symbol, "SMA", 50)
    sma200 = fetch_indicator(symbol, "SMA", 200)

    has_technicals = (not rsi.empty) or (not sma20.empty) or (not sma50.empty) or (not sma200.empty)
    lanes = choose_agents(has_news, has_prices, has_technicals)

    # 5) run agents
    outputs: List[AgentResponse] = []

    # NEWS (delegate summarization to the agent)
    if "news" in lanes and has_news:
        # Teammate agent expects `description`; remap our `summary` to `description`
        news_payload_records = (
            top_news
            .rename(columns={"summary": "description"})
            .loc[:, ["title", "description", "source", "url", "published_at"]]
            .to_dict(orient="records")
        )
        news_payload = {
            "ticker": symbol,
            "news": news_payload_records
        }
        outputs.append(NewsAnalysisAgent().process(news_payload))

    # TECHNICALS (build dict the agent expects; missing fields are fine)
    if "technical" in lanes and (has_technicals or has_prices):
        current_price = float(prices["close"].iloc[-1]) if has_prices else None
        volume = int(prices["volume"].iloc[-1]) if has_prices else None
        avg_volume = int(prices["volume"].tail(20).mean()) if has_prices else None

        technicals = {
            "current_price": current_price,
            "rsi": (float(rsi["RSI"].iloc[-1]) if not rsi.empty else None),
            # Map to teammate's keys; if we don't have these, leave None.
            "ma_50": (float(sma50["SMA"].iloc[-1]) if not sma50.empty else (float(sma20["SMA"].iloc[-1]) if not sma20.empty else None)),
            "ma_200": (float(sma200["SMA"].iloc[-1]) if not sma200.empty else None),
            "macd": None,                # not computed here
            "volume": volume,
            "avg_volume": avg_volume,
            "support": None,
            "resistance": None,
        }
        tech_payload = {
            "ticker": symbol,
            "technicals": technicals
        }
        outputs.append(MarketSignalsAgent().process(tech_payload))


    # RISK (simple proxy; fill what you have)
    vol_30d = float(prices["close"].pct_change().tail(30).std() * 100) if has_prices else None
    risk_payload = {
        "ticker": symbol,
        "risk_metrics": {
            "beta": None,
            "volatility": vol_30d,      # teammate agent expects 'volatility' (%)
            "var_5": None,
            "sharpe_ratio": None,
            "max_drawdown": None,
            "sector_correlation": None,
            "pe_ratio": None
        }
    }
    outputs.append(RiskAssessmentAgent().process(risk_payload))

    # 6) synthesize + critique (gated second pass)
    synth_v1 = SynthesisAgent().process(outputs)
    crit     = CritiqueAgent().process(synth_v1)

    # Decide if we need a second pass.
    # Rule: re-run if critique quality score < 0.90 OR if it mentions "data quality".
    needs_rerun = (crit.score < 0.90) or ("data quality" in " ".join(crit.key_factors).lower())

    synth_final = synth_v1
    if needs_rerun:
        # Turn critique into feedback for the optimizer pass
        critique_feedback = AgentResponse(
            agent_name="Critique Feedback",
            analysis=_as_text(synth_v1.analysis) + "\n\n[CRITIQUE]\n" + _as_text(crit.analysis),
            score=crit.score,
            confidence=crit.confidence,
            key_factors=_as_list_of_text(crit.key_factors),
            timestamp=datetime.now().isoformat()

        )

        synth_v2_inputs = outputs + [critique_feedback]
        synth_v2 = SynthesisAgent().process(synth_v2_inputs)
        synth_final = synth_v2

    # 7) memory (store both passes where applicable)
    append_memory({
        "ticker": symbol,
        "lanes": lanes,
        "issues": crit.key_factors,
        "final_confidence_v1": synth_v1.confidence,
        "final_confidence_v2": synth_final.confidence if needs_rerun else None,
        "optimizer_triggered": bool(needs_rerun),
        "timestamp": datetime.now().isoformat()
    })

    # 8) evidence for UI
    evidence = {
        "top_news": top_news,
        "prices_tail": prices.tail(5)
    }

    # Show the first synthesis alongside other agents for transparency
    outputs.append(AgentResponse(
        agent_name="Initial Synthesis",
        analysis=_as_text(synth_v1.analysis),
        score=float(synth_v1.score),
        confidence=float(synth_v1.confidence),
        key_factors=_as_list_of_text(synth_v1.key_factors),
        timestamp=synth_v1.timestamp
    ))

    return OrchestratorResult(plan, evidence, outputs, synth_final, crit)


# ============================================================================
# WORKFLOW PATTERN DEMONSTRATIONS
# ============================================================================

def run_prompt_chaining_workflow(symbol: str, start: str, end: str,
                                  required_tags: list[str] | None = None) -> AgentResponse:
    """
    WORKFLOW PATTERN 1: PROMPT CHAINING

    Demonstrates sequential processing where each step's output becomes
    the input to the next step, creating a progressive refinement chain.

    Pipeline: Ingest → Preprocess → Classify → Extract → Summarize

    Args:
        symbol: Stock ticker (e.g., 'AAPL')
        start: Start date (YYYY-MM-DD)
        end: End date (YYYY-MM-DD)
        required_tags: Optional tag filter

    Returns:
        AgentResponse with sentiment analysis
    """
    print("\n" + "="*80)
    print("WORKFLOW PATTERN 1: PROMPT CHAINING")
    print("="*80)
    print(f"Analyzing: {symbol} | Period: {start} to {end}")
    print("="*80)

    # Step 1: INGEST
    print("\n┌─ STEP 1/5: INGEST ─────────────────────────────────────────────┐")
    print("│ Fetching raw news from Alpha Vantage API...                    │")
    print("└─────────────────────────────────────────────────────────────────┘")
    raw_news = fetch_news(symbol)

    if raw_news.empty:
        print("  No news data available")
        return AgentResponse(
            agent_name="News Analysis Agent",
            analysis="No news data available",
            score=0.0,
            confidence=0.0,
            key_factors=["No data"],
            timestamp=datetime.now().isoformat()
        )

    print(f"  ✓ Fetched: {len(raw_news)} articles")

    # Step 2: PREPROCESS
    print("\n┌─ STEP 2/5: PREPROCESS ─────────────────────────────────────────┐")
    print("│ Cleaning, parsing dates, removing duplicates...                │")
    print("└─────────────────────────────────────────────────────────────────┘")
    clean_news = preprocess_news(raw_news)
    print(f"  ✓ Cleaned: {len(clean_news)} articles")

    # Step 3: CLASSIFY
    print("\n┌─ STEP 3/5: CLASSIFY ───────────────────────────────────────────┐")
    print("│ Tagging by topic, extracting numbers...                        │")
    print("└─────────────────────────────────────────────────────────────────┘")
    tagged_news = add_tags_and_numbers(clean_news)
    print(f"  ✓ Tagged: {len(tagged_news)} articles")

    # Step 4: EXTRACT
    print("\n┌─ STEP 4/5: EXTRACT ────────────────────────────────────────────┐")
    print("│ Filtering to top-K most relevant...                            │")
    print("└─────────────────────────────────────────────────────────────────┘")
    top_news = recent_topk(tagged_news, topk=SETTINGS.topk_news, days=SETTINGS.news_window_days, required_tags=required_tags)
    print(f"  ✓ Extracted: {len(top_news)} top articles")

    # Step 5: SUMMARIZE
    print("\n┌─ STEP 5/5: SUMMARIZE ──────────────────────────────────────────┐")
    print("│ LLM agent analyzing sentiment...                               │")
    print("└─────────────────────────────────────────────────────────────────┘")
    news_payload = {
        'ticker': symbol,
        'news': top_news.rename(columns={"summary": "description"}).to_dict('records') if not top_news.empty else []
    }
    result = NewsAnalysisAgent().process(news_payload)
    print(f"  ✓ Score: {result.score:+.2f} | Confidence: {result.confidence:.0%}")

    print("\n" + "="*80)
    print("PROMPT CHAINING COMPLETE")
    print("Pattern: Raw Data → Clean → Tagged → Filtered → Analysis")
    print("="*80 + "\n")

    return result


def run_parallel_workflow(symbol: str, start: str, end: str) -> List[AgentResponse]:
    """
    WORKFLOW PATTERN 2: PARALLEL EXECUTION

    Demonstrates concurrent execution of multiple agents for improved performance.
    Instead of sequential (A→B→C), runs simultaneously (A+B+C).

    Args:
        symbol: Stock ticker
        start: Start date
        end: End date

    Returns:
        List of AgentResponses from concurrent execution
    """
    from concurrent.futures import ThreadPoolExecutor
    import time

    print("\n" + "="*80)
    print("WORKFLOW PATTERN 2: PARALLEL EXECUTION")
    print("="*80)
    print(f"Analyzing: {symbol}")
    print("="*80)

    # Prepare data
    print("\n[Preparation] Fetching data...")
    news = fetch_news(symbol)
    prices = fetch_prices(symbol, start, end)

    # Prepare inputs
    news_input = {
        'ticker': symbol,
        'news': news.head(5).rename(columns={"summary": "description"}).to_dict('records') if not news.empty else []
    }

    tech_input = {
        'ticker': symbol,
        'technicals': {
            'current_price': float(prices['close'].iloc[-1]) if not prices.empty else None,
            'volume': int(prices['volume'].iloc[-1]) if not prices.empty else None,
        }
    }

    risk_input = {
        'ticker': symbol,
        'risk_metrics': {
            'volatility': float(prices["close"].pct_change().tail(30).std() * 100) if not prices.empty else None
        }
    }

    print("\n[Parallel] Running 3 agents concurrently...")
    print("  Sequential would be: Agent1 → Agent2 → Agent3")
    print("  Parallel runs: Agent1 + Agent2 + Agent3 simultaneously")

    start_time = time.time()

    # Execute in parallel
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {
            'news': executor.submit(NewsAnalysisAgent().process, news_input),
            'technical': executor.submit(MarketSignalsAgent().process, tech_input),
            'risk': executor.submit(RiskAssessmentAgent().process, risk_input)
        }

        results = {}
        for name, future in futures.items():
            results[name] = future.result()
            print(f"   {name.capitalize()}: Score={results[name].score:+.2f}")

    elapsed = time.time() - start_time

    print(f"\n" + "="*80)
    print(f"PARALLEL EXECUTION COMPLETE ({elapsed:.2f}s)")
    print(f"Speedup: ~2-3x faster than sequential execution")
    print("="*80 + "\n")

    return list(results.values())


def run_evaluator_optimizer_workflow(symbol: str, start: str, end: str,
                                      required_tags: list[str] | None = None) -> OrchestratorResult:
    """
    WORKFLOW PATTERN 3: EVALUATOR-OPTIMIZER (Self-Improving)

    Demonstrates adaptive refinement through Generate → Evaluate → Optimize loop.
    System critiques its own output and re-generates if quality is below threshold.

    This wraps the existing run_pipeline() to explicitly demonstrate the pattern.

    Args:
        symbol: Stock ticker
        start: Start date
        end: End date
        required_tags: Optional tag filter

    Returns:
        OrchestratorResult with potentially optimized synthesis
    """
    print("\n" + "="*80)
    print("WORKFLOW PATTERN 3: EVALUATOR-OPTIMIZER")
    print("="*80)
    print(f"Analyzing: {symbol}")
    print("="*80)

    # GENERATE
    print("\n[Phase 1] GENERATE: Running initial analysis...")
    result = run_pipeline(symbol, start, end, required_tags)

    initial_synthesis = next(
        (a for a in result.agent_outputs if "Initial Synthesis" in a.agent_name),
        None
    )

    if initial_synthesis:
        print(f"  Initial: Score={initial_synthesis.score:+.2f}, Conf={initial_synthesis.confidence:.0%}")

    # EVALUATE
    print(f"\n[Phase 2] EVALUATE: Critiquing quality...")
    print(f"  Quality Score: {result.critique.score:.2f}")
    print(f"  Issues Found: {len(result.critique.key_factors)}")

    # Check if optimizer ran
    optimizer_triggered = initial_synthesis and (initial_synthesis.analysis != result.final.analysis)

    # OPTIMIZE
    if optimizer_triggered:
        print(f"\n[Phase 3] OPTIMIZE: Re-synthesizing with feedback...")
        print(f"  Final: Score={result.final.score:+.2f}, Conf={result.final.confidence:.0%}")

        conf_change = result.final.confidence - initial_synthesis.confidence
        print(f"\n" + "="*80)
        print(f"OPTIMIZATION SUCCESSFUL")
        print(f"Confidence Change: {conf_change:+.0%} | Optimizer: YES")
        print("="*80 + "\n")
    else:
        print(f"\n" + "="*80)
        print(f"INITIAL QUALITY ACCEPTABLE")
        print(f"Quality: {result.critique.score:.2f} | Optimizer: NO")
        print("="*80 + "\n")

    return result


# ============================================================================
# DEMONSTRATION RUNNER
# ============================================================================

def demo_all_workflows(symbol: str = "AAPL"):
    """
    Run all three workflow patterns for demonstration purposes.

    This function is designed to be called from a notebook or script
    to show how the system implements different orchestration strategies.
    """
    from datetime import datetime, timedelta

    start = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
    end = datetime.now().strftime("%Y-%m-%d")

    print("\n" + "*"*40)
    print("  DEMONSTRATING 3 AGENTIC WORKFLOW PATTERNS")
    print("*"*40)
    print(f"\nTicker: {symbol}")
    print(f"Date Range: {start} to {end}\n")

    # Pattern 1
    result1 = run_prompt_chaining_workflow(symbol, start, end)

    # Pattern 2
    result2 = run_parallel_workflow(symbol, start, end)

    # Pattern 3
    result3 = run_evaluator_optimizer_workflow(symbol, start, end)

    print("\n" + "#"*40)
    print("  ALL 3 WORKFLOW PATTERNS DEMONSTRATED")
    print("#"*40 + "\n")

    return {
        'prompt_chaining': result1,
        'parallel': result2,
        'evaluator_optimizer': result3
    }


if __name__ == "__main__":
    # Run demonstration when file is executed directly
    demo_all_workflows("AAPL")



****************************************
  DEMONSTRATING 3 AGENTIC WORKFLOW PATTERNS
****************************************

Ticker: AAPL
Date Range: 2025-09-19 to 2025-10-19


WORKFLOW PATTERN 1: PROMPT CHAINING
Analyzing: AAPL | Period: 2025-09-19 to 2025-10-19

┌─ STEP 1/5: INGEST ─────────────────────────────────────────────┐
│ Fetching raw news from Alpha Vantage API...                    │
└─────────────────────────────────────────────────────────────────┘
  No news data available

WORKFLOW PATTERN 2: PARALLEL EXECUTION
Analyzing: AAPL

[Preparation] Fetching data...

[Parallel] Running 3 agents concurrently...
  Sequential would be: Agent1 → Agent2 → Agent3
  Parallel runs: Agent1 + Agent2 + Agent3 simultaneously
   News: Score=+0.00
   Technical: Score=+0.00
   Risk: Score=+0.50

PARALLEL EXECUTION COMPLETE (0.00s)
Speedup: ~2-3x faster than sequential execution


WORKFLOW PATTERN 3: EVALUATOR-OPTIMIZER
Analyzing: AAPL

[Phase 1] GENERATE: Running initial analysis...
  Initi

## ui/gradio_app.py

In [92]:
import os, sys
from pathlib import Path

# Works in both scripts and notebooks:
HERE = Path(__file__).parent if "__file__" in globals() else Path.cwd()
ROOT = (HERE / "..").resolve()
sys.path.append(str(ROOT))
import gradio as gr
import pandas as pd
from datetime import date, timedelta
from src.system.orchestrator import run_pipeline


def run(symbol, days_back, required_tags_csv):
    import json
    import pandas as pd

    # ---------- helpers ----------
    def as_text(x):
        if x is None:
            return ""
        if isinstance(x, str):
            return x
        if isinstance(x, (dict, list)):
            # pretty JSON for readability / stable comparisons
            return json.dumps(x, ensure_ascii=False, indent=2, sort_keys=True)
        return str(x)

    def clean(s: str) -> str:
        # normalize whitespace and strip code fences if any slipped through
        if not isinstance(s, str):
            s = as_text(s)
        s = s.strip()
        if s.startswith("```"):
            s = s.strip("`").strip()
        return s

    def synth_to_prose(obj):
        # If it's already text, just return normalized text
        if not isinstance(obj, dict):
            return clean(as_text(obj))

        parts = []

        ms = obj.get("market_signals") or {}
        if ms:
            ms_bits = []
            cp = ms.get("current_price")
            if isinstance(cp, (int, float)):
                ms_bits.append(f"price ${cp:,.2f}")
            ma = ms.get("moving_averages") or {}
            ma50 = ma.get("50_day")
            ma200 = ma.get("200_day")
            if ma50 is not None or ma200 is not None:
                ms_bits.append(f"vs 50D {ma50}, 200D {ma200}")
            rsi = ms.get("RSI")
            if rsi is not None:
                ms_bits.append(f"RSI {rsi}")
            trend = ms.get("trend")
            if trend:
                ms_bits.append(trend)
            vol = ms.get("volume") or {}
            vcur, vavg = vol.get("current"), vol.get("average")
            if vcur is not None and vavg is not None:
                ms_bits.append(f"volume {vcur:,} vs avg {vavg:,}")
            if ms_bits:
                parts.append("Technicals: " + ", ".join(str(x) for x in ms_bits if x))

        news = obj.get("news") or {}
        if news:
            # keep order stable-ish
            news_bits = []
            for k in ("sentiment", "growth potential", "competitive landscape"):
                if k in news:
                    news_bits.append(f"{k}: {news[k]}")
            # include any other keys if present
            for k, v in news.items():
                if k not in ("sentiment", "growth potential", "competitive landscape"):
                    news_bits.append(f"{k}: {v}")
            parts.append("News: " + "; ".join(news_bits))

        risk = obj.get("risk_assessment") or {}
        if risk:
            risk_bits = []
            for k in ("volatility", "data_gaps", "idiosyncratic_risks"):
                if k in risk:
                    risk_bits.append(f"{k}: {v}")
            for k, v in risk.items():
                if k not in ("volatility", "data_gaps", "idiosyncratic_risks"):
                    risk_bits.append(f"{k}: {v}")
            parts.append("Risk: " + "; ".join(risk_bits))

        return "\n".join(parts).strip()

    def to_df(x):
        if isinstance(x, pd.DataFrame):
            return x
        if x is None:
            return pd.DataFrame()
        try:
            return pd.DataFrame(x)
        except Exception:
            return pd.DataFrame()
    # ---------- /helpers ----------

    start = (date.today() - timedelta(days=int(days_back))).isoformat()
    end = date.today().isoformat()
    tags = [t.strip() for t in required_tags_csv.split(",")] if required_tags_csv else None

    res = run_pipeline(symbol.strip().upper(), start, end, required_tags=tags)

    # Detect if optimizer re-synthesis ran (compare Initial vs Final on normalized JSON text)
    optimizer_ran = False
    init = next((a for a in res.agent_outputs if a.agent_name in {"Initial Synthesis", "Research Synthesis Agent", "SynthesisAgent"}), None)
    if init is not None:
        init_txt = clean(as_text(init.analysis))
        final_txt_norm = clean(as_text(res.final.analysis))
        optimizer_ran = (init_txt != final_txt_norm)

    plan = "\n".join([f"• {step}" for step in res.plan])

    # Agents panel: Synthesis agents shown as prose; others as text
    agents_txt = "\n\n".join([
        (
            f"[{a.agent_name}] score={a.score:.2f} conf={a.confidence:.2f}\n"
            f"{synth_to_prose(a.analysis) if ('synthesis' in a.agent_name.lower()) else clean(as_text(a.analysis))}"
        )
        for a in res.agent_outputs
    ])

    # Evidence tables (force DataFrame)
    news_rows = to_df(res.evidence.get("top_news", []))
    prices_rows = to_df(res.evidence.get("prices_tail", []))

    # Helpful note when news evidence is empty
    if news_rows.empty:
        agents_txt += "\n\n[Note] No news items matched filters or API limits were hit today."

    # ---- Critique FIRST (so it appears above Final in the UI) ----
    crit_txt = (
        f"[Critique]\n"
        f"score={res.critique.score:.2f} adj_conf={res.critique.confidence:.2f}\n"
        f"{clean(as_text(res.critique.analysis))}"
    )

    # ---- Final AFTER Critique ----
    headline = "FINAL (After Critique)"
    opt_line = "[Optimizer ran: YES]" if optimizer_ran else "[Optimizer ran: NO]"
    final_txt = (
        f"{headline}\n{opt_line}\n"
        f"score={res.final.score:.2f} conf={res.final.confidence:.2f}\n"
        f"{synth_to_prose(res.final.analysis)}\n\nKey: {', '.join(res.final.key_factors)}"
    )

    # IMPORTANT: return order matches component outputs order (crit BEFORE final)
    return plan, agents_txt, crit_txt, final_txt, news_rows, prices_rows


with gr.Blocks(title="Agentic Finance") as demo:
    gr.Markdown("# Agentic Finance — Interactive Tester")

    with gr.Row():
        symbol = gr.Textbox(label="Ticker", value="AAPL")
        days_back = gr.Slider(7, 120, value=30, step=1, label="Days Back")
        tags = gr.Textbox(label="Required Tags (optional, comma-sep)", placeholder="earnings, product")
    run_btn = gr.Button("Run")

    plan = gr.Textbox(label="Plan", lines=6)
    agents = gr.Textbox(label="Agent Outputs", lines=14)

    #  Critique ABOVE Final
    crit = gr.Textbox(label="Critique", lines=8)
    final = gr.Textbox(label="Final Recommendation", lines=10)

    news_tbl = gr.Dataframe(
        headers=["published_at","source","title","summary","url","overall_sentiment","tags","numbers"],
        label="Top News (evidence)",
        wrap=True
    )
    prices_tbl = gr.Dataframe(label="Recent Prices (evidence)")

    # Outputs order: plan, agents, CRITIQUE, FINAL, news, prices
    run_btn.click(
        run,
        inputs=[symbol, days_back, tags],
        outputs=[plan, agents, crit, final, news_tbl, prices_tbl]
    )

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7dc5ab8312851ff8d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
